# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
#%matplotlib widget
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# from numba import cuda

# cuda.select_device(0) # choosing second GPU 
# cuda.close()

In [ ]:
import helper

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
norm_means = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]#
img_size = 224
batch_size = 64  # 32
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(norm_means, norm_std)
    ]),
    'valid': transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(norm_means, norm_std)
    ]),
}
data_transforms['test'] = data_transforms['valid']

# TODO: Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test':  datasets.ImageFolder(test_dir,  transform=data_transforms['test']),
}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=batch_size, shuffle=False),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False),
}

In [ ]:
# Run this to test your data loader
images, labels = next(iter(dataloaders['train']))
helper.imshow(images[0], normalize=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:
len(cat_to_name.keys())

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(cat_to_name, orient='index')

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# DONE: Build and train your network

In [ ]:
# Template class for the classifier network, inspired from
# https://github.com/udacity/DSND_Term1/blob/1196aafd48a2278b02eff85510b582fd7e2a9d2d/lessons/DeepLearning/new-intro-to-pytorch/fc_model.py
# Note: The usage of dropout didn't improve the performance with the given datasets - but required a few more training epochs to
# achieve the same performance.

class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.25):
        ''' Builds a feedforward network with arbitrary hidden layers.
        
            Arguments
            ---------
            input_size: integer, size of the input layer
            output_size: integer, size of the output layer
            hidden_layers: list of integers, the sizes of the hidden layers
        
        '''
        super().__init__()
        # Input to a hidden layer
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # Add a variable number of more hidden layers
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        # self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        for each in self.hidden_layers:
            x = F.relu(each(x))
            # x = self.dropout(x)
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)

### Usage of VGGx for training the classifer
Using VGGx as proposed provides good results.
Run the following cell to train using VGG11.

In [ ]:
model_name = 'vgg11'
model = getattr(models, model_name)(weights=models.VGG11_Weights.DEFAULT)

# Better use this because it could fit other image models better.
model_classifier = Network(25088, 102, [4096, 1024])

checkpoint_name = 'checkpoint_vgg11.pth'

In [ ]:
model.classifier[0].in_features

In [ ]:
model.classifier[0]

### Usage of Densenet for training the classifer
The usage of the much more simple ***densenet121*** provided good results for the classification of cats and dogs in the [transfer learning exercise](https://github.com/udacity/DSND_Term1/blob/1196aafd48a2278b02eff85510b582fd7e2a9d2d/lessons/DeepLearning/new-intro-to-pytorch/Part%208%20-%20Transfer%20Learning%20(Solution).ipynb) of this course. It turned out to perform well with flowers as well.
Run the following cell to train using densenet121.

In [ ]:
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

# Better use this because it could fit other image models better.
model_classifier = Network(1024, 102, [512])

checkpoint_name = 'checkpoint_densenet121.pth'

In [ ]:
#model.to('cpu')

### Run the following two cells to start training the classifier with the selected model

In [ ]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

model.classifier = model_classifier

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [ ]:
%%time
# This training loop was inspired by the "Tranfer learning exercise" too.
# https://github.com/udacity/DSND_Term1/blob/1196aafd48a2278b02eff85510b582fd7e2a9d2d/lessons/DeepLearning/new-intro-to-pytorch/Part%208%20-%20Transfer%20Learning%20(Solution).ipynb
# Note: In case of an "out of memory" of the GPU, try reducing the batch_size in the dataloaders.
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in dataloaders['train']:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in dataloaders['valid']:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(dataloaders['valid']):.3f}.. "
                  f"Test accuracy: {accuracy/len(dataloaders['valid']):.3f}")
            running_loss = 0
            model.train()

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set

In [ ]:
%%time

def test_model(model, dataloader):
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    print(f"Test loss: {test_loss/len(dataloaders['valid']):.3f}.. "
          f"Test accuracy: {accuracy/len(dataloaders['valid']):.3f}")
    running_loss = 0
    pass


test_model(model, dataloaders['test'])

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

In [ ]:
model.class_to_idx = image_datasets['train'].class_to_idx

In [ ]:
# Save the full model
checkpoint = {
    #'input_size': 784,
    #'output_size': 10,
    'model_module': type(model).__module__,
    'model_name': model_name,  # type(model).__name__,
    'state_dict': model.state_dict(),
    'class_to_idx': model.class_to_idx,
    'classifier': {
        'state_dict': model.classifier.state_dict(),
        'hidden_layers': [each.out_features for each in model.classifier.hidden_layers],
        'input_size': model.classifier.hidden_layers[0].in_features,
        'optimizer_state_dict': optimizer.state_dict(),
        #'loss': running_loss,
    },
}

torch.save(checkpoint, checkpoint_name)

In [ ]:
model.classifier.hidden_layers[0].in_features

In [ ]:
checkpoint['classifier']['hidden_layers']

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

In [ ]:
def load_checkpoint(filepath):
    import importlib

    checkpoint = torch.load(filepath)

    # Load the pre-trained model
    model_module = importlib.import_module(checkpoint['model_module'])
    print(checkpoint['model_name'])
    model = getattr(model_module, checkpoint['model_name'])()

    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False


    # TODO: load the pre-trained model
    model_classifier = Network(
        checkpoint['classifier']['input_size'],
        102,  # checkpoint['output_size'],
        checkpoint['classifier']['hidden_layers'])
    model.classifier = model_classifier

    model.load_state_dict(checkpoint['state_dict'])
    #model_classifier.load_state_dict(checkpoint['classifier']['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']

    # Optimizer state if we want to continue the training
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
    #optimizer.load_state_dict(checkpoint['classifier']['optimizer_state_dict'])

    return {'model': model, 'optimizer': optimizer}


In [ ]:
my_checkpoint = load_checkpoint(checkpoint_name)
model = my_checkpoint['model']
optimizer = my_checkpoint['optimizer']

criterion = nn.NLLLoss()


# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(model)
model.to(device);

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    img_transform = transforms.Compose([
            transforms.Resize(255),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(norm_means, norm_std)
        ])
    pil_img = Image.open(image)
    pt_img = img_transform(pil_img).float()

    return pt_img
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
from PIL import Image
import numpy as np

def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array(norm_means)
    std = np.array(norm_std)
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
imshow(process_image('sample_img.jpg'))

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.eval()
    pil_img = process_image(image_path)
    with torch.no_grad():
        input_img = pil_img.to(device)
        input_img.unsqueeze_(0)
        model.to(device)
        logps = model.forward(input_img)
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(topk, dim=1)
    idx_to_class = {model.class_to_idx[c]: c for c in model.class_to_idx.keys()}
    return top_p.cpu().numpy(), np.array([idx_to_class[idx] for idx in top_class.cpu().numpy()[0]])
# TODO: Implement the code to predict the class from an image file
    


In [ ]:
top_p, top_class = predict("sample_img.jpg", model)
print(top_p.squeeze().tolist())
print([cat_to_name[c] for c in top_class])

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

In [ ]:
def view_classify(image_path, model):
    
    ''' Function for viewing an image and it's predicted classes.
    '''
    top_p, top_class = predict(image_path, model)
    fig, (ax1, ax2) = plt.subplots(
        figsize=(5, 10), nrows=2,
    )

    imshow(process_image(image_path), ax=ax1)

    top_p_list = top_p.squeeze().tolist()
    top_class_names = [cat_to_name[c] for c in top_class]

    ax2.barh(np.arange(len(top_p_list)), top_p_list)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(len(top_p_list)))
    ax2.set_yticklabels(top_class_names)
    ax2.set_xlim(0, 1.1)



view_classify("canterbury_bells.jpg", model)